# Dask_traj

dask_traj is mdtraj, parallelized with dask to perform out-of-memory or out-of-machine computations
It lifts 3 important limitations of mdtraj:

1. The trajectory is to big to fit into memory at once: This could already be solved by mdtraj.iterload, but you still had to combine the results of the chunked calculations by hand
2. The result is to big to fit into memory: The results of dask-traj calculations are dask arrays, which know how to spill to disk.
3. Together with dask.distributed, this allows for mdtraj to scale from openMP (one machine) to MPI (multiple machines) which is usefull in HPC environments. (Discussed in the other example)

At the moment, not many functions are supported. But mdtraj functions can still be called on `dask_traj.Trajectory` after you called `.to_mdtraj()` on them (once) to convert them to a mdtraj.Trajectory. 

In [1]:
import mdtraj as md
import dask_traj as dmd
import itertools as itt
import numpy as np

In [2]:
# These files can be provided upon request, 
# but are to big to include in the repository (100 MB)
xtc = '5550217/new.xtc'
pdb = '5550217/new.pdb'
gro = '5550217/new.gro'

# Loading trajectories

There are 2 differences between `dmd.load()` and `md.load()`.
1. `dmd.load()` has a keyword `chunks` that is the number of frames per chunk (naming was copied from dask)
2. `dmd.load()` is a delayed function, so it does not actually load the files (it just checks the lenght of the file, so a filetype where this is known, like .xtc, would be adviced)

In [3]:
%time dask_traj = dmd.load(xtc, top=gro, chunks=100)

CPU times: user 737 ms, sys: 20.6 ms, total: 757 ms
Wall time: 751 ms


In [4]:
%time mdtraj = md.load(xtc, top=gro)

CPU times: user 1.8 s, sys: 65.4 ms, total: 1.86 s
Wall time: 1.85 s


In [5]:
mdtraj

<mdtraj.Trajectory with 1251 frames, 22561 atoms, 6832 residues, and unitcells at 0x7f2d3690ce80>

In [6]:
dask_traj

<mdtraj.Trajectory with 1251 frames, 22561 atoms, 6832 residues, and unitcells at 0x7f2d1aa0e080>

In [7]:
np.testing.assert_array_equal(mdtraj.xyz, dask_traj.xyz)

In [8]:
# This will fail as MDTraj has an issue where it is losing accuracy on the loaded unitcell_vectors
# see https://github.com/mdtraj/mdtraj/issues/1463
# np.testing.assert_array_equal(mdtraj.unitcell_vectors, dask_traj.unitcell_vectors)

# Doing a computation

In [9]:
# Make some atom pairs
c = list(itt.combinations(range(500),2))
# This does not work anymore for mdtraj (Memory error, depending on the RAM size)
# but should still work for dask_traj
#c = list(itt.combinations(range(3000),2))
print(len(c))

124750


In [10]:
%%time 
# This makes the computation graph, but does not compute anything yet
dmd_result = dmd.compute_distances(dask_traj, atom_pairs=c)

CPU times: user 85.7 ms, sys: 218 µs, total: 85.9 ms
Wall time: 84.5 ms


In [11]:
# We can visualize the computation graph that will be performed.
dmd_result.visualize(optimize_graph=True)

`mdtraj` is heavily optimized, so we do not expect this to be faster (also because `dask_traj` is just now loading the file)

In [12]:
%%time 
dmd_out = dmd_result.compute()

CPU times: user 13.2 s, sys: 139 ms, total: 13.4 s
Wall time: 13.3 s


In [13]:
%%time
md_result = md.compute_distances(mdtraj, atom_pairs=c)

CPU times: user 11.5 s, sys: 50.9 ms, total: 11.5 s
Wall time: 11.5 s


# Compare the results

In [14]:
# The tollerance is needed due to the round-tripping of unitcell_vectors by mdtraj, but not by dask-traj 
# (see https://github.com/mdtraj/mdtraj/issues/1463)
np.testing.assert_allclose(md_result, dmd_out, atol=2e-6)

# A fairer timing comparison
Here we time all the code (loading+computation) instead of one at a time to estimate the overhead.
Please also have a look at the distributed example notebook. 

In [15]:
%%time
dask_traj = dmd.load(xtc, top=gro, chunks=100)
dmd_result = dmd.compute_distances(dask_traj, atom_pairs=c).compute()

CPU times: user 14 s, sys: 193 ms, total: 14.2 s
Wall time: 14.1 s


In [16]:
%%time
mdtraj = md.load(xtc, top=gro)
md_result = md.compute_distances(mdtraj, atom_pairs=c)

CPU times: user 13.5 s, sys: 99.9 ms, total: 13.6 s
Wall time: 13.5 s


# Not implemented functions

`compute_dihedrals` is not implemented at the moment, but we can still call the `mdtraj` function after using the `to_mdtraj` function.

In [17]:
# As mdtraj.ensure_type does not know about dask.arrays (at the moment) we need to convert our trajectory
dask_traj.to_mdtraj() # This will transform the whole trajectory to a mdtraj.Trajectory

In [18]:
indices = [(1,2,3,4),(5,6,7,8)]
dmd_result = md.compute_dihedrals(dask_traj, indices=indices)
md_result = md.compute_dihedrals(mdtraj, indices=indices)

In [19]:
np.testing.assert_allclose(md_result, dmd_result, atol=2e-6)

In [20]:
# This now passes as the trajectory is truely cast to a mdtraj.Trajectory, with the cycling of unitcell_vectors.
np.testing.assert_array_equal(mdtraj.unitcell_vectors, dask_traj.unitcell_vectors)